# Creating and Valuing a CDS Index Option

This notebook does the valuation of an option on a CDS index using Anderson's method and an Adjusted Black Method. For details see Modelling Singlename and Multiname Credit Derivatives by D.O'Kane.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.33 - This build:  11 Nov 2023 at 07:47 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



In [3]:
value_date = Date(1, 8, 2007)
step_in_date = value_date.add_weekdays(0)
value_date = step_in_date
settle_date = step_in_date

## Build Libor Curve

In [4]:
dcType = DayCountTypes.THIRTY_E_360_ISDA
depos = []
depo = IborDeposit(settle_date, "1D", 0.0500, dcType); depos.append(depo)

fixedFreq = FrequencyTypes.SEMI_ANNUAL
swapType = SwapTypes.PAY
swap1 = IborSwap(settle_date,"1Y",swapType,0.0502,fixedFreq,dcType)
swap2 = IborSwap(settle_date,"2Y",swapType,0.0502,fixedFreq,dcType)
swap3 = IborSwap(settle_date,"3Y",swapType,0.0501,fixedFreq,dcType)
swap4 = IborSwap(settle_date,"4Y",swapType,0.0502,fixedFreq,dcType)
swap5 = IborSwap(settle_date,"5Y",swapType,0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

libor_curve = IborSingleCurve(value_date, depos, [], swaps)

In [5]:
cdsMaturity3Y = value_date.next_cds_date(36)
cdsMaturity5Y = value_date.next_cds_date(60)
cdsMaturity7Y = value_date.next_cds_date(84)
cdsMaturity10Y = value_date.next_cds_date(120)

## Load the Underlying CDS Index Portfolio

In [6]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
issuer_curves = []

num_credits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    creditName = splitRow[0]
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recovery_rate = float(splitRow[5])

    cds3Y = CDS(step_in_date, cdsMaturity3Y, spd3Y)
    cds5Y = CDS(step_in_date, cdsMaturity5Y, spd5Y)
    cds7Y = CDS(step_in_date, cdsMaturity7Y, spd7Y)
    cds10Y = CDS(step_in_date, cdsMaturity10Y, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]

    issuer_curve = CDSCurve(value_date, cds_contracts, libor_curve, recovery_rate)
    issuer_curves.append(issuer_curve)

## Term Structure of CDS Index Market

We now set up the term structure of CDS Index trades and start by setting up the maturity dates of the index

In [7]:
indexMaturity3Y = Date(20,12,2009)
indexMaturity5Y = Date(20,12,2011)
indexMaturity7Y = Date(20,12,2013)
indexMaturity10Y = Date(20,12,2016)

Now we set their upfronts and coupons

In [8]:
index_upfronts = [0.0, 0.0, 0.0, 0.0]
index_maturity_dates = [indexMaturity3Y, indexMaturity5Y, indexMaturity7Y, indexMaturity10Y]
indexRecovery = 0.40

## Setting up the Index Option

In [9]:
expiry_date = Date(1,2,2008)
maturity_date = Date(20,12,2011)
notional = 10000
volatility = 0.50
index_coupon = 0.004
tolerance = 1e-6

## Comparison of Anderson versus Adjusted Black Model

In [10]:
print("STRIKE    INDEX      PAY    RECEIVER      G(K)       X         EXPH       ABPAY       ABREC")

for index in np.linspace(20, 60, 5):
    
    # Create a flat CDS index curve
    cds_contracts = []
    for dt in index_maturity_dates:
        cds = CDS(value_date, dt, index / 10000.0)
        cds_contracts.append(cds)

    # Build the flat CDS index curve
    index_curve = CDSCurve(value_date, cds_contracts, libor_curve, indexRecovery)

    # Now we need to set up the underlying curves and adjust them
    indexSpreads = [index / 10000.0] * 4

    indexPortfolio = CDSIndexPortfolio()
    adjustedIssuerCurves = indexPortfolio.hazard_rate_adjust_intrinsic(value_date, 
                                                                    issuer_curves, 
                                                                    indexSpreads, 
                                                                    index_upfronts,
                                                                    index_maturity_dates, 
                                                                    indexRecovery, 
                                                                    tolerance)        
        
    for strike in np.linspace(20, 60, 5):    

        option = CDSIndexOption(expiry_date,
                                   maturity_date, 
                                   index_coupon, 
                                   strike / 10000.0, 
                                   notional)
        
        v_pay_1, v_rec_1, strike_value, mu, expH = option.value_anderson(value_date, 
                                                                       adjustedIssuerCurves, 
                                                                       indexRecovery,
                                                                       volatility)

        v_pay_2, v_rec_2 = option.value_adjusted_black(value_date,
                                                     index_curve, 
                                                     indexRecovery, 
                                                     libor_curve,
                                                     volatility)
            
        print("%6.3f   %6.3f   %8.3f   %8.3f   %8.3f   %8.2f   %8.2f   %8.2f   %8.2f" % 
              (strike, index, v_pay_1, v_rec_1, strike_value, mu, expH, v_pay_2, v_rec_2 ))

STRIKE    INDEX      PAY    RECEIVER      G(K)       X         EXPH       ABPAY       ABREC
20.000   20.000     16.090      6.237    -70.799      22.87     -60.70      16.12       6.15
30.000   20.000      4.040     28.882    -35.287      22.85     -60.76       4.04      28.73
40.000   20.000      0.885     60.202      0.000      22.83     -60.82       0.90      59.98
50.000   20.000      0.188     93.764     35.064      22.82     -60.88       0.20      93.40
60.000   20.000      0.043    127.661     69.907      22.80     -60.94       0.05     127.08
20.000   30.000     50.337      0.896    -70.799      34.35     -20.11      50.40       0.84
30.000   30.000     24.031      9.255    -35.287      34.34     -20.14      24.07       9.21
40.000   30.000      9.749     29.420      0.000      34.33     -20.17       9.83      29.38
50.000   30.000      3.654     57.554     35.064      34.32     -20.20       3.72      57.41
60.000   30.000      1.325     89.237     69.907      34.31     -20.23 

FinError: Max Iterations exceeded

In [ ]:
print(option)

OBJECT TYPE: CDSIndexOption
EXPIRY DATE: 01-FEB-2008
MATURITY DATE: 20-DEC-2011
INDEX COUPON: 40.0bp
NOTIONAL: 10000
LONG PROTECTION: True
FREQUENCY: FrequencyTypes.QUARTERLY
DAYCOUNT: DayCountTypes.ACT_360
CALENDAR: CalendarTypes.WEEKEND
BUSDAYRULE: BusDayAdjustTypes.FOLLOWING
DATEGENRULE: DateGenRuleTypes.BACKWARD



Copyright (c) 2020 Dominic O'Kane